In [1]:
import xml.etree.ElementTree as ET
import regex as re
import json
from tqdm import tqdm

In [3]:
def findGender_Birth(desc):
    reg = r'\[\[Kategori:Födda[^\n]*'
    regnum = r'\p{N}+'

    gender = ""
    if "[[Kategori:Män]]" in desc or "[[Kategori:Kvinnor]]" in desc:  
        if "[[Kategori:Män]]" in desc:
            gender = "Man"
        else:
            gender = "Woman"
    
    birth = re.finditer(reg,child.findall('revision')[0].findall('text')[0].text)
    temp = birth
    birth = ""
    for b in temp:
        birth = b.group()
    year = ""
    if len(birth) > 0:
        year = re.findall(regnum,birth)

    return gender,year,birth

In [4]:
def findRef(desc):
    regref = r'<ref(.*?)</ref>'
    regtext = r'.*?=\s?Källor\s?='
    
    text = re.search(regtext,desc,re.DOTALL)

    refrences = None
    if text != None:
        refrences = re.finditer(regref,text.group())
    refrence = []
    
    if refrences != None:
        for ref in refrences:
            if "nordisk familjebok" in ref.group().lower() or "runeberg.org|nf" in ref.group().lower():
                refrence.append(ref.group())

    return refrence

In [5]:
def findSrc(desc,refrence):
    regsource = r'=+\s*Källor\s*=+\s*(.*?)(?===+)'
    source = re.search(regsource,desc,re.DOTALL)
    sources = []
    if source != None:
        source = source.group()
        source = source.splitlines()
        for l in source:
            if "nordisk familjebok" in l.lower() or "runeberg.org|nf" in l.lower():
                sources.append(l) 
            if "<references/>" in l.lower():
                if len(refrence) > 0:
                    for r in refrence:
                        sources.append(r)
    else:
        regsource = r'=+\s*Källor\s*=+\s*(.*)'
        source = re.search(regsource,desc,re.DOTALL)
        if source != None:
            source = source.group()
            source = source.splitlines()
            for l in source:
                if "nordisk familjebok" in l.lower() or "runeberg.org|nf" in l.lower():
                    sources.append(l)
                if "<references/>" in l.lower().replace(" ",""):
                    if len(refrence) > 0:
                        for r in refrence:
                            sources.append(r)

    return sources

In [6]:
def descClean(name,desc):
    reg = rf"'''{name.split()[0]}[^\n]*"
    descInd = re.search(reg,desc)

    if descInd != None:
        desc = descInd.group(0)
    else:
        desc = ""
    desc = desc.replace("'''","")
    desc = desc.replace("''","")
    desc = desc.replace("[[","")
    desc = desc.replace("]]","")
    desc = desc.replace("|"," ")
    desc= re.sub(r"<[^>]+>.*?</[^>]+>", "", desc, flags=re.DOTALL)
    desc = re.sub(r'<(.*?)\/>',"",desc,flags=re.DOTALL)
    desc= re.sub(r"\{\{.*?\}\}", "", desc, flags=re.DOTALL)
    desc = desc.replace("()","")

    return desc

In [8]:
for z in tqdm(range(283)):
    tree = ET.parse(rf'chunk_{z}_done.xml')
    root = tree.getroot()
    
    i = 0
    final = []
    for child in root:
        if len(child.findall('revision')[0].findall('text')) > 0:
            
            desc = child.findall('revision')[0].findall('text')[0].text
            if desc != None:

                gender,year,birth = findGender_Birth(desc)

                if gender != "":
                    if len(year) > 0:
                        if int(year[0]) < 1956:
                            i += 1

                            refrence = findRef(desc)
                            
                            sources = findSrc(desc,refrence)
                    
                            name = child.findall('title')[0].text
                            desc = descClean(name,desc)
                            
                            final.append({'name':name,'description':desc,'birth':year[0],'knowBirthDate':1,'gender':gender,'refrences':refrence,'sources':sources})
                    else:
                        i += 1

                        refrence = findRef(desc)
                            
                        sources = findSrc(desc,refrence)
                
                        name = child.findall('title')[0].text
                        desc = descClean(name,desc)
                      
                        final.append({'name':name,'description':desc,'birth':birth,'knowBirthDate':0,'gender':gender,'refrences':refrence,'sources':sources})
                        

    with open(rf'wiki_data_{z}.json','w',encoding='utf-8') as f:
        json.dump(final,f,indent = 4,ensure_ascii = False)
        f.close()
    
    print(i)

  0%|          | 1/283 [00:02<11:05,  2.36s/it]

1309


  1%|          | 2/283 [00:04<10:22,  2.21s/it]

1769


  1%|          | 3/283 [00:06<09:47,  2.10s/it]

...

2503


100%|█████████▉| 282/283 [07:35<00:02,  2.09s/it]

1859


100%|██████████| 283/283 [07:37<00:00,  1.62s/it]

1413
